In [1]:
from pycalphad import Database
from pycalphad.eq.energy_surf import energy_surf
import numpy as np
import sys
db_nbre = Database('nbre_liu.tdb')
my_phases_nbre = ['CHI_RENB', 'SIGMARENB', 'FCC_RENB', 'LIQUID_RENB', 'BCC_RENB', 'HCP_RENB']
nrg_df = energy_surf(db_nbre, ['RE', 'NB'], my_phases_nbre, T=np.linspace(300, 3000, num=300), pdens=5000)
rsize = nrg_df.values.nbytes / 2**20.
print("Size for regular values in dataframe: %.1f MB" % (rsize,))
dfo = nrg_df.blocks['object']
import sys
overhead = sum(sys.getsizeof(o) for o in dfo.values.flat) / 2**20.
print("Overhead of objects: %.1f MB" % (overhead ,))
dfsize = rsize + overhead
print("Total size for the dataframe: %.1f MB" % (dfsize,))
print(nrg_df.dtypes)

Size for regular values in dataframe: 400.7 MB
Overhead of objects: 960.3 MB
Total size for the dataframe: 1361.0 MB
GM       float64
Phase     object
T        float64
X(NB)    float64
X(RE)    float64
dtype: object


In [2]:
import bcolz
nrg_df.columns = [c.replace('(', '_').replace(',', '_').replace(')', '') for c in nrg_df.columns]
%time zdf = bcolz.ctable.fromdataframe(nrg_df)
zsize = zdf.cbytes / 2**20.
print("Total size for the ctable: %.1f MB" % (zsize,))
# Compression ratio
print("Compression ratio the ctable: %.1fx" % (zdf.nbytes / float(zdf.cbytes),))
# Reduction in size compared with the dataframe
print("Storage savings compared with a dataframe: %.1fx" % (dfsize / zsize,))

RuntimeError: fatal error during Blosc compression: -1